In [9]:
!pip install matplotlib
!pip install seaborn
!pip install pandas
!pip install numpy
!pip install pyspark


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 

cast_path = "../data/actor_actress_director.csv"
cast_df = pd.read_csv(
    cast_path,
    low_memory=False
)

conn = sqlite3.connect('../data/cast.db')

cast_df.to_sql('actor_actress_director', conn, index=False, if_exists='replace')

result = pd.read_sql_query("select * from actor_actress_director limit 5;", conn)
print(result)

conn.close()

      tconst     nconst        primaryName  category titleType  \
0  tt0000009  nm0183823  William Courtenay     actor     movie   
1  tt0000009  nm1309758     Chauncey Depew     actor     movie   
2  tt0000009  nm0063086    Blanche Bayliss   actress     movie   
3  tt0000009  nm0085156    Alexander Black  director     movie   
4  tt0000574  nm0846894          John Tait     actor     movie   

                  primaryTitle  isAdult  releaseYear  releaseMonth  \
0                   Miss Jerry        0         1894            10   
1                   Miss Jerry        0         1894            10   
2                   Miss Jerry        0         1894            10   
3                   Miss Jerry        0         1894            10   
4  The Story of the Kelly Gang        0         1906            12   

  releaseDate  ...  tmdb_averageRating  tmdb_numVotes  revenue  budget  \
0  1894-10-08  ...               5.000              3        0       0   
1  1894-10-08  ...               5

In [11]:
print(cast_df.dtypes)

tconst                     object
nconst                     object
primaryName                object
category                   object
titleType                  object
primaryTitle               object
isAdult                     int64
releaseYear                 int64
releaseMonth                int64
releaseDate                object
runtime_consolidated      float64
imdb_averageRating        float64
imdb_numVotes             float64
tmdb_averageRating        float64
tmdb_numVotes               int64
revenue                     int64
budget                      int64
popularity                float64
weighted_averageRating    float64
total_numVotes            float64
bayesian_averageRating    float64
profit_ratio              float64
aaa_index                 float64
dtype: object


In [18]:
# Find the most active actors, actresses, and directors in 2020

conn = sqlite3.connect('../data/cast.db')

active_cast_query = """
    with base as (
        select
            primaryName,
            nconst,
            category,
            count(distinct tconst) as num_starred_movies
        from actor_actress_director
        where 1=1
            and releaseYear = 2020
        group by 1,2,3
    )

    select
        primaryName,
        category,
        num_starred_movies,
        rn
    from (
        select
            primaryName,
            category,
            num_starred_movies,
            row_number() over (partition by category order by num_starred_movies desc) as rn
        from base
    )
    where 1=1
        and rn = 1
    order by 2, 3 desc
"""

active_cast_result = pd.read_sql(active_cast_query, conn)
print(active_cast_result)

       primaryName  category  num_starred_movies  rn
0     Eric Roberts     actor                  14   1
1      Sheri Davis   actress                  10   1
2  Dustin Ferguson  director                  12   1


In [13]:
# find the most popular cast members in 2020 based on average popularity score

popular_cast_query = """ 
    with base as (
        select
            primaryName,
            nconst,
            category,
            avg(popularity) as avg_popularity
        from actor_actress_director
        where 1=1
            and releaseYear = 2020
            and popularity > 0
        group by 1,2,3
    )

    select
        primaryName,
        category,
        avg_popularity
    from (
        select
            primaryName,
            category,
            avg_popularity,
            row_number() over (partition by category order by avg_popularity desc) as rn
        from base
    )
    where 1=1
        and rn = 1
    order by 2, 3 desc
"""

popular_cast_result = pd.read_sql(popular_cast_query, conn)
print(popular_cast_result)

       primaryName  category  avg_popularity
0      Mike Hatton     actor         227.981
1     Mara Fimbres   actress         227.981
2  Christian Sesma  director         227.981


In [14]:
# find the most popular cast members in 2020 based on total review votes

votes_cast_query = """
    with base as (
        select
            primaryName,
            nconst,
            category,
            sum(total_numVotes) as total_votes
        from actor_actress_director
        where 1=1 
            and releaseYear = 2020
        group by 1,2,3
    )

    select
        primaryName,
        category,
        total_votes
    from (
        select
            primaryName,
            category,
            total_votes,
            row_number() over (partition by category order by total_votes desc) as rn
        from base
    )
    where 1=1
        and rn = 1
    order by 2, 3 desc
"""

votes_cast_result = pd.read_sql(votes_cast_query, conn)
print(votes_cast_result)

         primaryName  category  total_votes
0    Rich Ceraulo Ko     actor     635705.0
1  Elizabeth Debicki   actress     629342.0
2  Christopher Nolan  director     629342.0


In [20]:
lt_votes_cast_query = """
    with base as (
        select
            primaryName,
            nconst,
            category,
            sum(total_numVotes) as total_votes
        from actor_actress_director
        where 1=1 
            and releaseYear between 2020 and 2025
        group by 1,2,3
    )

    select
        primaryName,
        category,
        total_votes
    from (
        select
            primaryName,
            category,
            total_votes,
            row_number() over (partition by category order by total_votes desc) as rn
        from base
    )
    where 1=1
        and rn between 1 and 5
    order by 2, 3 desc
"""

lt_votes_cast_result = pd.read_sql(lt_votes_cast_query, conn)
print(lt_votes_cast_result)

          primaryName  category  total_votes
0           John Cena     actor    5535238.0
1        Ben Schwartz     actor    4911249.0
2          Kevin Hart     actor    4788110.0
3          Greg Kriek     actor    4475402.0
4      Brandon Morgan     actor    4462665.0
5   Nathalie Emmanuel   actress    4917573.0
6      Melissa Ponzio   actress    4526872.0
7       Alon McKlveen   actress    4464802.0
8          Paula Pell   actress    4462665.0
9             Zendaya   actress    2667839.0
10         Eric Appel  director    4513363.0
11   Denis Villeneuve  director    1534484.0
12  Christopher Nolan  director    1479970.0
13         Shawn Levy  director    1167155.0
14          Jon Watts  director     949383.0


In [16]:
conn.close()